In [1]:
import sympy as sp
import numpy as np
from sympy import latex

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from modules.ur3_symbols import *
from modules.kinematics import get_forward_kinematics, get_transform, build_symbolic_com_jacobians
from modules.dynamics import (compute_inertia_matrices, compute_coriolis_matrix,
                              compute_com_wrt_base, compute_potential_energy, compute_gravity_torque_vector)

In [2]:
# this is the official dh table from universal robotics, not the one derived by Coppelia's frame assignments

ur3_dh_table = [
    {'theta':q_syms[0], 'a':0, 'd':d_syms[0], 'alpha':sp.pi/2},
    {'theta':q_syms[1], 'a':a_syms[1], 'd':0, 'alpha':0},
    {'theta':q_syms[2], 'a':a_syms[2], 'd':0, 'alpha':0},
    {'theta':q_syms[3], 'a':0, 'd':d_syms[3], 'alpha':sp.pi/2},
    {'theta':q_syms[4], 'a':0, 'd':d_syms[4], 'alpha':-sp.pi/2},
    {'theta':q_syms[5], 'a':0, 'd':d_syms[5], 'alpha':0},
]

ur3_subs = {
    d_syms[0]: 0.1519,
    a_syms[1]: -0.24365,
    a_syms[2]: -0.21325,
    d_syms[3]: 0.11235,
    d_syms[4]: 0.08535,
    d_syms[5]: 0.0819
}

ur3_com_subs = {
r_c1x: 0,
r_c1y: -0.02,
r_c1z: 0,
r_c2x: 0.13,
r_c2y: 0,
r_c2z: 0.1157,
r_c3x: 0.05,
r_c3y: 0,
r_c3z: 0.0238,
r_c4x: 0,
r_c4y: 0,
r_c4z: 0.01,
r_c5x: 0,
r_c5y: 0,
r_c5z: 0.01,
r_c6x: 0,
r_c6y: 0,
r_c6z: -0.02,
}

n = len(ur3_dh_table)

In [3]:
ur3_transforms = get_forward_kinematics(ur3_dh_table)
ur3_transforms = [sp.trigsimp(transform) for transform in ur3_transforms]

In [4]:
# Extract the necessary components for the jacobian
origins = [ H[:3, -1] for H in ur3_transforms ]
z_vectors = [ H[:3, 2] for H in ur3_transforms[1:]] # the first transform is not needed since it's just the identity
center_of_masses = [r_c1, r_c2, r_c3, r_c4, r_c5, r_c6]

In [5]:
jacobians = build_symbolic_com_jacobians(origins, z_vectors, center_of_masses)
jacobians[0]

Matrix([
[                   -r_{c_1,z}*cos(q1), 0, 0, 0, 0, 0],
[                   -r_{c_1,z}*sin(q1), 0, 0, 0, 0, 0],
[r_{c_1,x}*cos(q1) + r_{c_1,y}*sin(q1), 0, 0, 0, 0, 0]])

In [6]:
inertia_matrices = compute_inertia_matrices(jacobians[4:], m_syms[4:])

In [7]:
# sum all of the inertia matrices for each link
full_inertia_mat = sum(inertia_matrices, sp.zeros(*inertia_matrices[0].shape))

In [8]:
# do not try to print the full matrix
coriolis = compute_coriolis_matrix(full_inertia_mat, q_syms, dq_syms)


In [9]:
# do not include the first transform which is just an identity
center_of_masses_wrt_base = compute_com_wrt_base(center_of_masses, ur3_transforms[1:])
g_vec = sp.Matrix([0, 0, -g]) #positive z is up
U_q = compute_potential_energy(g_vec, m_syms, center_of_masses_wrt_base)
g_q = compute_gravity_torque_vector(U_q, q_syms)
g_q

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      0],
[-g*m2*(a2*cos(q2) + r_{c_2,x}*cos(q2) - r_{c_2,y}*sin(q2)) - g*m3*(a2*cos(q2) + a3*cos(q2 + q3) + r_{c_3,x}*cos(q2 + q3) - r_{c_3,y}*sin(q2 + q3)) - g*m4*(a2*cos(q2) + a3*cos(q2 + q3) + r_{c_4,x}*cos(q2 + q3 + q4) + r_{c_4,z}*sin(q2 + q3 + q4)) - g*m5*(a2*cos(q2) + a3*cos(q2 + q3) + d5*sin(